In [ ]:
# Tải và giải nén dữ liệu MovieLens
!wget https://files.grouplens.org/datasets/moviellens/ml-latest-small.zip -q
!unzip -q ml-latest-small.zip
print("Tải và giải nén dữ liệu xong.")

replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace ml-latest-small/tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace ml-latest-small/ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace ml-latest-small/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace ml-latest-small/movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Tải và giải nén dữ liệu xong.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr
import warnings

# Bỏ qua các cảnh báo RuntimeWarning (ví dụ: khi tính Pearson cho vector hằng)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

def tai_va_chuan_bi_du_lieu():
    """Tải và chuẩn bị dữ liệu từ tệp ratings.csv."""
    print("Đang tải và chuẩn bị dữ liệu...")
    try:
        ratings = pd.read_csv('ml-latest-small/ratings.csv')
    except FileNotFoundError:
        print("Lỗi: Không tìm thấy tệp 'ml-latest-small/ratings.csv'.")
        print("Vui lòng tải và giải nén bộ dữ liệu MovieLens 100k trước.")
        return None, None, None

    # Giảm kích thước dữ liệu để chạy nhanh hơn (ví dụ: 1000 người dùng đầu tiên)
    # Bỏ dòng này nếu muốn chạy trên toàn bộ 100k ratings
    # common_users = ratings['userId'].value_counts().nlargest(1000).index
    # ratings = ratings[ratings['userId'].isin(common_users)]

    # Chia dữ liệu
    train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=42)

    # Tạo ma trận user-item từ dữ liệu huấn luyện
    # Hàng: userId, Cột: movieId, Giá trị: rating
    train_matrix = train_df.pivot(
        index='userId',
        columns='movieId',
        values='rating'
    )
    return train_matrix, test_df

def tinh_tuong_dong_pearson(u1_ratings, u2_ratings):
    """Tính độ tương đồng Pearson giữa hai vector rating (pandas Series)."""

    # Tìm các item mà cả hai user đều đã đánh giá
    common_items = (u1_ratings.notna()) & (u2_ratings.notna())

    # Nếu có ít hơn 2 item chung, không thể tính correlation -> 0
    if common_items.sum() < 2:
        return 0.0

    # Lấy rating của các item chung
    u1_common = u1_ratings[common_items]
    u2_common = u2_ratings[common_items]

    # Tính Pearson correlation
    corr, _ = pearsonr(u1_common, u2_common)

    # Xử lý trường hợp NaN (ví dụ: nếu rating của 1 user là hằng số)
    if np.isnan(corr):
        return 0.0

    return corr

def tinh_tuong_dong_euclidean(u1_ratings, u2_ratings):
    """Tính độ tương đồng Euclidean giữa hai vector rating (pandas Series)."""

    # Tìm các item mà cả hai user đều đã đánh giá
    common_items = (u1_ratings.notna()) & (u2_ratings.notna())

    # Nếu không có item chung -> 0
    if common_items.sum() == 0:
        return 0.0

    # Lấy rating của các item chung
    u1_common = u1_ratings[common_items]
    u2_common = u2_ratings[common_items]

    # Tính khoảng cách Euclidean
    dist = euclidean(u1_common, u2_common)

    # Chuyển đổi khoảng cách (distance) thành độ tương đồng (similarity)
    # Giá trị càng gần 1 càng tốt (khi khoảng cách = 0)
    return 1 / (1 + dist)

def tao_ma_tran_tuong_dong(train_matrix, similarity_func):
    """Tính toán và trả về ma trận tương đồng user-user."""
    n_users = train_matrix.shape[0]
    user_ids = train_matrix.index

    # Tạo ma trận rỗng để lưu trữ độ tương đồng
    sim_matrix = pd.DataFrame(
        np.zeros((n_users, n_users)),
        index=user_ids,
        columns=user_ids
    )

    print(f"Đang tính ma trận tương đồng cho {n_users} người dùng...")

    # Tính toán ma trận tam giác (đối xứng)
    for i in range(n_users):
        for j in range(i, n_users):
            user_i_id = user_ids[i]
            user_j_id = user_ids[j]

            if i == j:
                sim = 1.0
            else:
                sim = similarity_func(
                    train_matrix.loc[user_i_id],
                    train_matrix.loc[user_j_id]
                )

            sim_matrix.loc[user_i_id, user_j_id] = sim
            sim_matrix.loc[user_j_id, user_i_id] = sim

        if (i+1) % 100 == 0:
            print(f"Đã xử lý {i+1}/{n_users} người dùng.")

    return sim_matrix

def du_doan_rating(user_id, item_id, train_matrix, user_means, similarity_matrix, k=30):
    """Dự đoán rating cho 1 user-item, sử dụng K-Nearest Neighbors."""

    # Lấy rating trung bình của user mục tiêu
    # Nếu user mới, dùng rating trung bình toàn cục
    global_mean = user_means.mean()
    target_user_mean = user_means.get(user_id, global_mean)

    # Lấy vector tương đồng của user_id với tất cả user khác
    user_sims = similarity_matrix.loc[user_id]

    # Lấy tất cả ratings cho item_id
    item_ratings = train_matrix[item_id]

    # Lọc ra những user đã rating item_id này
    valid_neighbors = item_ratings.dropna()

    # Lấy độ tương đồng của các "hàng xóm" hợp lệ này
    neighbor_sims = user_sims.loc[valid_neighbors.index]

    # Loại bỏ chính user_id (nếu có)
    neighbor_sims = neighbor_sims.drop(user_id, errors='ignore')

    # Sắp xếp và chọn K hàng xóm gần nhất
    top_k_neighbors = neighbor_sims.nlargest(k)

    # --- Tính toán dự đoán theo công thức Mean-Centered ---
    # Prediction(u, i) = mean(u) + [ sum( sim(u, v) * (r(v, i) - mean(v)) ) / sum( |sim(u, v)| ) ]

    weighted_sum = 0.0
    sim_sum = 0.0

    for neighbor_id, sim in top_k_neighbors.items():
        # Lấy rating và rating trung bình của hàng xóm
        neighbor_rating = valid_neighbors[neighbor_id]
        neighbor_mean = user_means[neighbor_id]

        # Công thức
        weighted_sum += sim * (neighbor_rating - neighbor_mean)
        sim_sum += abs(sim)

    # Xử lý chia cho 0 (không có hàng xóm hợp lệ)
    if sim_sum == 0:
        prediction = target_user_mean
    else:
        prediction = target_user_mean + (weighted_sum / sim_sum)

    # Đảm bảo rating nằm trong khoảng 0.5 - 5.0 (theo chuẩn MovieLens)
    prediction = np.clip(prediction, 0.5, 5.0)

    return prediction

def danh_gia_mo_hinh(test_df, train_matrix, user_means, similarity_matrix):
    """Đánh giá mô hình trên tập test và trả về MSE."""

    # Lọc test_df để chỉ giữ lại user và item đã có trong tập train
    known_users = test_df['userId'].isin(train_matrix.index)
    known_items = test_df['movieId'].isin(train_matrix.columns)
    test_set = test_df[known_users & known_items]

    print(f"Đánh giá trên {len(test_set)} mẫu test hợp lệ...")

    predictions = []
    actuals = []

    for _, row in test_set.iterrows():
        user_id = row['userId']
        item_id = row['movieId']
        actual_rating = row['rating']

        pred_rating = du_doan_rating(
            user_id,
            item_id,
            train_matrix,
            user_means,
            similarity_matrix
        )

        predictions.append(pred_rating)
        actuals.append(actual_rating)

    # Tính toán Mean Squared Error
    mse = mean_squared_error(actuals, predictions)
    return mse

# --- Hàm MAIN để chạy ---
def main():
    train_matrix, test_df = tai_va_chuan_bi_du_lieu()

    if train_matrix is None:
        return

    # Tính rating trung bình cho mỗi user (để chuẩn hóa)
    user_means = train_matrix.mean(axis=1)

    # --- 1. Đánh giá với PEARSON CORRELATION ---
    print("\n--- Bắt đầu với Pearson Correlation ---")
    sim_matrix_pearson = tao_ma_tran_tuong_dong(
        train_matrix,
        tinh_tuong_dong_pearson
    )
    mse_pearson = danh_gia_mo_hinh(
        test_df,
        train_matrix,
        user_means,
        sim_matrix_pearson
    )
    print(f"==> MSE (Pearson): {mse_pearson:.4f}")

    # --- 2. Đánh giá với EUCLIDEAN DISTANCE ---
    print("\n--- Bắt đầu với Euclidean Similarity ---")
    sim_matrix_euclidean = tao_ma_tran_tuong_dong(
        train_matrix,
        tinh_tuong_dong_euclidean
    )
    mse_euclidean = danh_gia_mo_hinh(
        test_df,
        train_matrix,
        user_means,
        sim_matrix_euclidean
    )
    print(f"==> MSE (Euclidean): {mse_euclidean:.4f}")

    # --- 3. Kết luận ---
    print("\n--- KẾT QUẢ SO SÁNH ---")
    print(f"MSE (Pearson):   {mse_pearson:.4f}")
    print(f"MSE (Euclidean): {mse_euclidean:.4f}")

    if mse_pearson < mse_euclidean:
        print("Pearson Correlation cho kết quả tốt hơn (MSE thấp hơn).")
    else:
        print("Euclidean Similarity cho kết quả tốt hơn (MSE thấp hơn).")

if __name__ == "__main__":
    main()

Đang tải và chuẩn bị dữ liệu...

--- Bắt đầu với Pearson Correlation ---
Đang tính ma trận tương đồng cho 610 người dùng...
Đã xử lý 100/610 người dùng.
Đã xử lý 200/610 người dùng.
Đã xử lý 300/610 người dùng.
Đã xử lý 400/610 người dùng.
Đã xử lý 500/610 người dùng.
Đã xử lý 600/610 người dùng.
Đánh giá trên 19355 mẫu test hợp lệ...
==> MSE (Pearson): 0.7956

--- Bắt đầu với Euclidean Similarity ---
Đang tính ma trận tương đồng cho 610 người dùng...
Đã xử lý 100/610 người dùng.
Đã xử lý 200/610 người dùng.
Đã xử lý 300/610 người dùng.
Đã xử lý 400/610 người dùng.
Đã xử lý 500/610 người dùng.
Đã xử lý 600/610 người dùng.
Đánh giá trên 19355 mẫu test hợp lệ...
==> MSE (Euclidean): 0.8322

--- KẾT QUẢ SO SÁNH ---
MSE (Pearson):   0.7956
MSE (Euclidean): 0.8322
Pearson Correlation cho kết quả tốt hơn (MSE thấp hơn).
